In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1171957,2021-04-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1171958,2021-04-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1171959,2021-04-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1171960,2021-04-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
31423,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31425,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31427,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31429,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31431,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
722366,2021-04-11,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804
722368,2021-04-12,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804
722370,2021-04-13,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804
722372,2021-04-14,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1171957,2021-04-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1171958,2021-04-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1171959,2021-04-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1171960,2021-04-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-15') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
450,2021-04-15,Snohomish,Washington,34114,564.00,53061,WA,County,822083,4149.70,68.61
898,2021-04-15,Cook,Illinois,517808,10357.00,17031,IL,County,5150233,10054.07,201.10
1345,2021-04-15,Orange,California,268622,4869.00,6059,CA,County,3175692,8458.69,153.32
1791,2021-04-15,Maricopa,Arizona,529722,9750.00,4013,AZ,County,4485414,11809.88,217.37
2237,2021-04-15,Los Angeles,California,1228176,23579.00,6037,CA,County,10039107,12233.92,234.87
...,...,...,...,...,...,...,...,...,...,...,...
1171345,2021-04-15,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1171530,2021-04-15,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1171684,2021-04-15,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1171834,2021-04-15,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
450,2021-04-15,Snohomish,Washington,34114,564.00,53061,WA,County,822083,4149.70,68.61,68.61,4149.70
898,2021-04-15,Cook,Illinois,517808,10357.00,17031,IL,County,5150233,10054.07,201.10,201.10,10054.07
1345,2021-04-15,Orange,California,268622,4869.00,6059,CA,County,3175692,8458.69,153.32,153.32,8458.69
1791,2021-04-15,Maricopa,Arizona,529722,9750.00,4013,AZ,County,4485414,11809.88,217.37,217.37,11809.88
2237,2021-04-15,Los Angeles,California,1228176,23579.00,6037,CA,County,10039107,12233.92,234.87,234.87,12233.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171345,2021-04-15,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1171530,2021-04-15,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1171684,2021-04-15,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1171834,2021-04-15,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
755309,2021-04-15,Hale,Alabama,2186,75.00,1065,AL,County,14651,14920.48,511.91,511.91,14920.48,1
822079,2021-04-15,Clarke,Alabama,3467,60.00,1025,AL,County,23622,14677.00,254.00,254.00,14677.00,2
591081,2021-04-15,Lowndes,Alabama,1365,53.00,1085,AL,County,9726,14034.55,544.93,544.93,14034.55,3
439866,2021-04-15,Franklin,Alabama,4205,82.00,1059,AL,County,31362,13407.95,261.46,261.46,13407.95,4
524136,2021-04-15,Etowah,Alabama,13677,346.00,1055,AL,County,102268,13373.68,338.33,338.33,13373.68,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111749,2021-04-15,Platte,Wyoming,617,11.00,56031,WY,County,8393,7351.36,131.06,131.06,7351.36,19
720841,2021-04-15,Converse,Wyoming,990,17.00,56009,WY,County,13822,7162.49,122.99,122.99,7162.49,20
895986,2021-04-15,Lincoln,Wyoming,1385,12.00,56023,WY,County,19830,6984.37,60.51,60.51,6984.37,21
997052,2021-04-15,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
591081,2021-04-15,Lowndes,Alabama,1365,53.00,1085,AL,County,9726,14034.55,544.93,544.93,14034.55,3,1
755309,2021-04-15,Hale,Alabama,2186,75.00,1065,AL,County,14651,14920.48,511.91,511.91,14920.48,1,2
251515,2021-04-15,Walker,Alabama,7126,275.00,1127,AL,County,63521,11218.34,432.93,432.93,11218.34,28,3
637398,2021-04-15,Greene,Alabama,913,34.00,1063,AL,County,8111,11256.32,419.18,419.18,11256.32,27,4
522588,2021-04-15,Clay,Alabama,1500,55.00,1027,AL,County,13235,11333.59,415.56,415.56,11333.59,24,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635857,2021-04-15,Hot Springs,Wyoming,364,3.00,56017,WY,County,4413,8248.36,67.98,67.98,8248.36,17,19
895986,2021-04-15,Lincoln,Wyoming,1385,12.00,56023,WY,County,19830,6984.37,60.51,60.51,6984.37,21,20
873684,2021-04-15,Uinta,Wyoming,2155,12.00,56041,WY,County,20226,10654.60,59.33,59.33,10654.60,4,21
250336,2021-04-15,Teton,Wyoming,3689,9.00,56039,WY,County,23464,15721.96,38.36,38.36,15721.96,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169206,2021-04-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1169207,2021-04-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1169208,2021-04-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1169209,2021-04-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
752177,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,1.00
752178,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
752179,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
752180,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
752181,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,14920.48,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910950,2021-04-11,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00
910951,2021-04-12,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00
910952,2021-04-13,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,3.00
910953,2021-04-14,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
588713,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,1.00,0.00
588714,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
588715,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
588716,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
588717,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586778,2021-04-11,Albany,Wyoming,4055,11.00,56001,WY,County,38880,10429.53,28.29,28.29,10558.13,23,5,0.00,0.00
586779,2021-04-12,Albany,Wyoming,4079,11.00,56001,WY,County,38880,10491.26,28.29,28.29,10558.13,23,5,24.00,0.00
586780,2021-04-13,Albany,Wyoming,4093,11.00,56001,WY,County,38880,10527.26,28.29,28.29,10558.13,23,5,14.00,0.00
586781,2021-04-14,Albany,Wyoming,4096,11.00,56001,WY,County,38880,10534.98,28.29,28.29,10558.13,23,5,3.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-15') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
300215,2021-04-15,Imperial,California,27907,717.00,6025,CA,County,181215,15399.94,395.66,395.66,15399.94,1,2,20.00,0.00
2237,2021-04-15,Los Angeles,California,1228176,23579.00,6037,CA,County,10039107,12233.92,234.87,234.87,12233.92,2,5,550.00,26.00
149547,2021-04-15,San Bernardino,California,294262,4523.00,6071,CA,County,2180085,13497.73,207.47,207.47,13497.73,3,4,161.00,2.00
525636,2021-04-15,Inyo,California,1392,36.00,6027,CA,County,18039,7716.61,199.57,199.57,7716.61,4,25,1.00,0.00
66901,2021-04-15,Stanislaus,California,60245,1026.00,6099,CA,County,550660,10940.51,186.32,186.32,10940.51,5,9,116.00,3.00
31017,2021-04-15,Riverside,California,297078,4519.00,6065,CA,County,2470546,12024.79,182.92,182.92,12024.79,6,6,170.00,5.00
82938,2021-04-15,Tulare,California,49355,827.00,6107,CA,County,466195,10586.77,177.39,177.39,10586.77,7,10,37.00,2.00
53235,2021-04-15,San Joaquin,California,71338,1334.00,6077,CA,County,762148,9360.12,175.03,175.03,9360.12,8,19,109.00,0.00
30207,2021-04-15,Fresno,California,100376,1640.00,6019,CA,County,999101,10046.63,164.15,164.15,10046.63,9,14,19.00,4.00
476377,2021-04-15,Merced,California,31151,452.00,6047,CA,County,277680,11218.31,162.78,162.78,11218.31,10,8,23.00,2.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1111300,2021-04-15,Lassen,California,5666,1.00,6035,CA,County,30573,18532.69,3.27,3.27,18532.69,56,1,0.00,0.00
300215,2021-04-15,Imperial,California,27907,717.00,6025,CA,County,181215,15399.94,395.66,395.66,15399.94,1,2,20.00,0.00
637340,2021-04-15,Kings,California,22737,245.00,6031,CA,County,152940,14866.61,160.19,160.19,14866.61,11,3,7.00,0.00
149547,2021-04-15,San Bernardino,California,294262,4523.00,6071,CA,County,2180085,13497.73,207.47,207.47,13497.73,3,4,161.00,2.00
2237,2021-04-15,Los Angeles,California,1228176,23579.00,6037,CA,County,10039107,12233.92,234.87,234.87,12233.92,2,5,550.00,26.00
31017,2021-04-15,Riverside,California,297078,4519.00,6065,CA,County,2470546,12024.79,182.92,182.92,12024.79,6,6,170.00,5.00
188001,2021-04-15,Kern,California,107648,1311.00,6029,CA,County,900202,11958.20,145.63,145.63,11958.20,14,7,57.00,5.00
476377,2021-04-15,Merced,California,31151,452.00,6047,CA,County,277680,11218.31,162.78,162.78,11218.31,10,8,23.00,2.00
66901,2021-04-15,Stanislaus,California,60245,1026.00,6099,CA,County,550660,10940.51,186.32,186.32,10940.51,5,9,116.00,3.00
82938,2021-04-15,Tulare,California,49355,827.00,6107,CA,County,466195,10586.77,177.39,177.39,10586.77,7,10,37.00,2.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
300215,2021-04-15,Imperial,California,27907,717.00,6025,CA,County,181215,15399.94,395.66,395.66,15399.94,1,2,20.00,0.00
2237,2021-04-15,Los Angeles,California,1228176,23579.00,6037,CA,County,10039107,12233.92,234.87,234.87,12233.92,2,5,550.00,26.00
149547,2021-04-15,San Bernardino,California,294262,4523.00,6071,CA,County,2180085,13497.73,207.47,207.47,13497.73,3,4,161.00,2.00
525636,2021-04-15,Inyo,California,1392,36.00,6027,CA,County,18039,7716.61,199.57,199.57,7716.61,4,25,1.00,0.00
66901,2021-04-15,Stanislaus,California,60245,1026.00,6099,CA,County,550660,10940.51,186.32,186.32,10940.51,5,9,116.00,3.00
31017,2021-04-15,Riverside,California,297078,4519.00,6065,CA,County,2470546,12024.79,182.92,182.92,12024.79,6,6,170.00,5.00
82938,2021-04-15,Tulare,California,49355,827.00,6107,CA,County,466195,10586.77,177.39,177.39,10586.77,7,10,37.00,2.00
53235,2021-04-15,San Joaquin,California,71338,1334.00,6077,CA,County,762148,9360.12,175.03,175.03,9360.12,8,19,109.00,0.00
30207,2021-04-15,Fresno,California,100376,1640.00,6019,CA,County,999101,10046.63,164.15,164.15,10046.63,9,14,19.00,4.00
476377,2021-04-15,Merced,California,31151,452.00,6047,CA,County,277680,11218.31,162.78,162.78,11218.31,10,8,23.00,2.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,395.66,15399.94,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,395.66,15399.94,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,395.66,15399.94,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,395.66,15399.94,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,395.66,15399.94,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3932,04/11/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,0.00,0.00
3933,04/12/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,0.00,0.00
3934,04/13/21,Lassen,5666,1.00,18532.69,3.27,3.27,18532.69,56,1,6.00,0.00
3935,04/14/21,Lassen,5666,1.00,18532.69,3.27,3.27,18532.69,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,395.66,15399.94,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,395.66,15399.94,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,395.66,15399.94,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,395.66,15399.94,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,395.66,15399.94,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4018,04/11/21,Merced,31009,448.00,11167.17,161.34,162.78,11218.31,10,8,0.00,0.00
4019,04/12/21,Merced,31070,448.00,11189.14,161.34,162.78,11218.31,10,8,61.00,0.00
4020,04/13/21,Merced,31105,449.00,11201.74,161.70,162.78,11218.31,10,8,35.00,1.00
4021,04/14/21,Merced,31128,450.00,11210.03,162.06,162.78,11218.31,10,8,23.00,1.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)